# Exemplos de tools funcionais

Agora vamos abordar a criação de duas tools com utilidade real para você entender como seria todo o processo.

## Criando um tool de busca de temperatura

https://open-meteo.com/en/docs

In [3]:
import requests
import datetime

from langchain.agents import tool
from langchain.pydantic_v1 import BaseModel, Field

class RetornTempArgs(BaseModel):
    latitude: float = Field(description='Latitude da localidade que buscamos a temperatura')
    longitude: float = Field(description='Longitude da localidade que buscamos a temperatura')


@tool(args_schema=RetornTempArgs)
def retorna_temperatura_atual(latitude: float, longitude: float):
    '''Retorna a temperatura atual para uma dada coordenada'''

    URL = 'https://api.open-meteo.com/v1/forecast'

    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    resposta = requests.get(URL, params=params)
    if resposta.status_code == 200:
        resultado = resposta.json()
        
        hora_agora = datetime.datetime.now(datetime.UTC).replace(tzinfo=None)
        lista_horas = [datetime.datetime.fromisoformat(temp_str) for temp_str in resultado['hourly']['time']]
        index_mais_prox = min(range(len(lista_horas)), key=lambda x: abs(lista_horas[x] - hora_agora))

        temp_atual = resultado['hourly']['temperature_2m'][index_mais_prox]
        return temp_atual
    else:
        raise Exception(f'Request para API {URL} falhou: {resposta.status_code}')


In [4]:
retorna_temperatura_atual

StructuredTool(name='retorna_temperatura_atual', description='retorna_temperatura_atual(latitude: float, longitude: float) - Retorna a temperatura atual para uma dada coordenada', args_schema=<class '__main__.RetornTempArgs'>, func=<function retorna_temperatura_atual at 0x0000018C57F1CA40>)

In [5]:
retorna_temperatura_atual.args

{'latitude': {'title': 'Latitude',
  'description': 'Latitude da localidade que buscamos a temperatura',
  'type': 'number'},
 'longitude': {'title': 'Longitude',
  'description': 'Longitude da localidade que buscamos a temperatura',
  'type': 'number'}}

### Testando a tool

In [7]:
retorna_temperatura_atual.invoke({'latitude': -30, 'longitude': -50})

20.3

## Criando um tool de busca no wikipedia

In [18]:
import wikipedia
wikipedia.set_lang('pt')

@tool
def busca_wikipedia(query: str):
    """Faz busca no wikipedia e retorna resumos de páginas para a query"""
    titulos_paginas = wikipedia.search(query)
    resumos = []
    for titulo in titulos_paginas[:3]:
        try:
            wiki_page = wikipedia.page(title=titulo, auto_suggest=True)
            resumos.append(f'Título da página: {titulo}\nResumo: {wiki_page.summary}')
        except:
            pass
    if not resumos:
        return 'Busca não teve retorno'
    else:
        return '\n\n'.join(resumos)

In [19]:
busca_wikipedia.invoke({'query': 'futebol'})

'Título da página: Futebol\nResumo: O futebol, também referido como futebol de campo, futebol de onze e, originalmente, futebol associado (em inglês: association football, football, soccer), é um esporte de equipe jogado entre dois times de 11 jogadores cada um e um árbitro que se ocupa da correta aplicação das normas. É considerado o desporto mais popular do mundo, pois cerca de 270 milhões de pessoas participam das suas várias competições. É jogado num campo retangular gramado, com uma baliza em cada lado do campo. O objetivo do jogo é deslocar uma bola através do campo para colocá-la dentro da baliza adversária, ação que se denomina golo (português europeu) ou gol (português brasileiro). A equipe que marca mais gols ao término da partida é a vencedora.\nO jogo moderno foi criado na Inglaterra com a formação de The Football Association, cujas regras de 1863 são a base do desporto na atualidade. O órgão regente do futebol é a Federação Internacional de Futebol (em francês: Fédération 

### Testando a tool

In [21]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function

prompt = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável chamado Isaac'),
    ('user', '{input}')
])
chat = ChatOpenAI()

tools = [convert_to_openai_function(busca_wikipedia), convert_to_openai_function(retorna_temperatura_atual)]
chain = prompt | chat.bind(functions=tools)


In [22]:
chain.invoke({'input': 'Olá'})

AIMessage(content='Olá! Como posso ajudar você hoje?', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 148, 'total_tokens': 160}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-81757f8e-4e55-44e8-aac4-9f0f1d8aa09f-0')

In [23]:
chain.invoke({'input': 'Qual é a temperatura de Porto Alegre?'})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":-30.0346,"longitude":-51.2177}', 'name': 'retorna_temperatura_atual'}}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 156, 'total_tokens': 184}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-962073d5-6bd1-4d75-86fc-40f9535288d8-0')

In [24]:
chain.invoke({'input': 'Quem foi Isaac Asimov?'})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Isaac Asimov"}', 'name': 'busca_wikipedia'}}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 154, 'total_tokens': 174}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-1c6dd1a6-a47b-4c5d-887b-beca183fa0a9-0')